In [ ]:
# install packages
!pip install catboost
!pip install xgboost
!pip install lightgbm
!pip install mljar-supervised

In [2]:
# import packages
import os
import numpy as np
import pandas as pd

# mljar
from supervised.automl import AutoML

import catboost
import lightgbm
import xgboost

from sklearn.preprocessing import LabelEncoder
from joblib import load, dump

In [3]:
# # connect with Google Cloud
# from google.colab import drive
# drive.mount('/content/drive')
# path = "/content/drive/My Drive/colab/TPS Jun"
path = r'C:\Users\Chen\Desktop\Kaggle\Classifier\Tabular Playground Series - Jun 2021'
os.chdir(path)

In [4]:
# load data
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [5]:
# target labelling
f = LabelEncoder()
f.fit(df_train['target'])
y = f.transform(df_train['target'])
y = pd.DataFrame(y).astype('int64')

col = [i for i in df_train.columns if i not in ['id', 'target']]
x = df_train[col]
x = x.astype('int64')

In [8]:
# construct model
# AutoML : mljar
RESULTS_PATH = 'mljar-20210608'
SEED = 20210608

cv = {"validation_type": "kfold",
      "k_folds": 5,
      "shuffle": True,
      "stratify": True,
      "random_seed": SEED}

automl = AutoML(results_path=RESULTS_PATH,
                mode="Optuna",                          # or 'Explain', 'Perform', 'Compete'
                ml_task='multiclass_classification',   # or 'auto', 'binary_classification', 'regression'
                algorithms=['Baseline', 'Linear', 'Decision Tree', 'Random Forest', 'Extra Trees', 'LightGBM', 'Xgboost', 'CatBoost', 'Nearest Neighbors'],   # 'Neural Network'
                train_ensemble=True,
                stack_models=True,
                eval_metric='logloss',
                validation_strategy=cv,
                golden_features=True,
                boost_on_errors=True,
                optuna_time_budget=int(1.5*60*60),
                total_time_limit=int(12*60*60),
                optuna_verbose=False,
                n_jobs=-1,
                random_state=SEED)

In [9]:
# training model
automl.fit(x, y)

Linear algorithm was disabled.
AutoML directory: mljar-20210608
Expected computing time:
Total training time: Optuna + ML training = 86400 seconds
Total Optuna time: len(algorithms) * optuna_time_budget = 43200 seconds
Total ML model training time: 43200 seconds
The task is multiclass_classification with evaluation metric logloss
AutoML will use algorithms: ['Baseline', 'Decision Tree', 'Random Forest', 'Extra Trees', 'LightGBM', 'Xgboost', 'CatBoost', 'Nearest Neighbors']
AutoML will stack models
AutoML will ensemble availabe models
AutoML steps: ['simple_algorithms', 'default_algorithms', 'golden_features', 'boost_on_errors', 'ensemble', 'stack', 'ensemble_stacked']
* Step simple_algorithms will try to check up to 2 models
1_Baseline logloss 1.897468 trained in 18.45 seconds
2_DecisionTree logloss 1.81761 trained in 27.25 seconds
* Step default_algorithms will try to check up to 5 models
3_Optuna_LightGBM logloss 1.749792 trained in 1945.2 seconds
Data has no positive values, and the

AutoML(algorithms=['Baseline', 'Linear', 'Decision Tree', 'Random Forest',
                   'Extra Trees', 'LightGBM', 'Xgboost', 'CatBoost',
                   'Nearest Neighbors'],
       boost_on_errors=True, eval_metric='logloss', golden_features=True,
       ml_task='multiclass_classification', mode='Optuna',
       optuna_time_budget=5400, optuna_verbose=False, random_state=20210608,
       results_path='mljar-20210608', stack_models=True, total_time_limit=43200,
       validation_strategy={'k_folds': 5, 'random_seed': 20210608,
                            'shuffle': True, 'stratify': True,
                            'validation_type': 'kfold'})

In [10]:
# load well-trained model
automl = AutoML(results_path=RESULTS_PATH)

In [11]:
# predict probability
x_test = df_test.drop(['id'], axis=1)
x_test = x_test.astype('int64')
result = automl.predict_proba(x_test)

In [12]:
# submission
sub = pd.read_csv('sample_submission.csv')
sub[sub.columns[1:]] = result
sub.to_csv(f'{RESULTS_PATH}.csv', index=False)